<a href="https://colab.research.google.com/github/eunShim/BigDataCapstone/blob/main/bigdatacapstone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- KT4 파일로 500명 샘플링해서 사용자 로그 quit 횟수, timestamp 차이(소요시간), mobile/web 비율, submit 비율 계산함
- 500명 행동 패턴 통계 출력

In [1]:
# ============================================
# ✅ EdNet-KT4.zip 500명 샘플 분석 + 시각화 (7z 버전)
# ============================================
import os, random, pandas as pd, numpy as np, matplotlib.pyplot as plt
from tqdm import tqdm

# --------------------------------------------
# 1. 7z 설치 및 내부 파일 목록 확인
# --------------------------------------------
!sudo apt-get install -y p7zip-full > /dev/null

zip_path = "/content/EdNet-KT4.zip"
list_output = os.popen(f"7z l {zip_path} | grep '.csv' | awk '{{print $6}}'").read().splitlines()

if not list_output:
    raise RuntimeError("❌ CSV 파일 목록을 찾을 수 없습니다. 파일이 손상되었거나 7z 포맷이 아닐 수 있습니다.")

print(f"✅ 내부 CSV 파일 {len(list_output)}개 탐색됨")

# --------------------------------------------
# 2. 무작위 500명 샘플링 후 추출
# --------------------------------------------
sample_files = random.sample(list_output, 500)
os.makedirs("/content/KT4_sample", exist_ok=True)

# 선택된 500명만 추출
for f in tqdm(sample_files, desc="📦 압축 해제 중"):
    os.system(f"7z x '{zip_path}' '{f}' -o'/content/KT4_sample' -y > /dev/null")

print("✅ 샘플 500명 추출 완료!")

# --------------------------------------------
# 3. 사용자별 행동 통계 계산
# --------------------------------------------
folder = "/content/KT4_sample"
files = [os.path.join(folder, f) for f in os.listdir(folder) if f.endswith(".csv")]

user_stats = []

for f in tqdm(files, desc="📊 사용자 분석 중"):
    try:
        df = pd.read_csv(f)
        if df.empty or "timestamp" not in df.columns:
            continue

        user_id = os.path.basename(f).replace(".csv", "")
        df = df.sort_values("timestamp")

        quit_count = (df["action_type"] == "quit").sum()
        duration = df["timestamp"].iloc[-1] - df["timestamp"].iloc[0]
        mobile_ratio = (df["platform"] == "mobile").mean()
        web_ratio = (df["platform"] == "web").mean()
        submit_count = (df["action_type"] == "submit").sum()
        unique_q = df[df["item_id"].astype(str).str.startswith("q")]["item_id"].nunique()
        submit_ratio = submit_count / unique_q if unique_q > 0 else np.nan

        user_stats.append({
            "user_id": user_id,
            "quit_count": quit_count,
            "duration_ms": duration,
            "mobile_ratio": mobile_ratio,
            "web_ratio": web_ratio,
            "submit_ratio": submit_ratio
        })

    except Exception as e:
        print(f"⚠️ {f} 처리 오류:", e)

stats_df = pd.DataFrame(user_stats)
print("\n✅ 사용자별 통계 미리보기:")
print(stats_df.head())

# --------------------------------------------
# 4. 통계 요약
# --------------------------------------------
print("\n📈 전체 통계 요약:")
print(stats_df.describe())

# --------------------------------------------
# 5. 시각화
# --------------------------------------------
plt.figure(figsize=(8,4))
stats_df["quit_count"].hist(bins=30, color="lightcoral")
plt.title("Quit 행동 횟수 분포")
plt.xlabel("quit_count")
plt.ylabel("사용자 수")
plt.show()

plt.figure(figsize=(8,4))
stats_df["submit_ratio"].hist(bins=30, color="mediumseagreen")
plt.title("문항 수 대비 Submit 비율 분포")
plt.xlabel("submit_ratio")
plt.ylabel("사용자 수")
plt.show()

plt.figure(figsize=(6,4))
stats_df[["mobile_ratio","web_ratio"]].mean().plot(kind='bar', color=["orange","skyblue"])
plt.title("평균 플랫폼 비율 (Mobile vs Web)")
plt.ylabel("비율")
plt.show()

plt.figure(figsize=(8,4))
plt.scatter(stats_df["duration_ms"]/60000, stats_df["quit_count"], alpha=0.6, c="slateblue")
plt.title("사용자별 소요시간 vs Quit 횟수")
plt.xlabel("소요시간(분)")
plt.ylabel("Quit 횟수")
plt.show()

print("\n✅ 분석 및 시각화 완료! DataFrame 이름: stats_df")


RuntimeError: ❌ CSV 파일 목록을 찾을 수 없습니다. 파일이 손상되었거나 7z 포맷이 아닐 수 있습니다.